In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
from scipy.optimize import curve_fit

In [ ]:
##
#Define Path to Code Database
DirPath = '/Your/Path/To/Code/'

##
#Define Output Path
OutputPath = '/Your/Path/To/Output/'

In [ ]:
##
#Load Functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from MotionSimulation import *
from EPGForwardMotionModel import *
from Jacobian_MotionCorrection import *
from ParameterOptionsSimulation import *

In [ ]:
##
#Read Parameters
opt = ParameterOptionsSimulation()

In [ ]:
##
#Conventional EPG simulates a maximum k-value pathway equal to the number of TRs. This piece of code identifies where increasing the k-value leads to no meaningful change in the signal, with subsequent thresholding to accelerate modelling
opt["kOrder"] = kOrder(opt.copy())

In [ ]:
##
#Define Motion Operators - Translation Velocity (mm/s), Rotation Velocity (deg/s), and Maximum Cardiac Velocity (mm/s) along x,y & z-axes
Trans= np.array([0, 0, 0])
Rot = np.array([0, 0, 0])
Card = np.array([0, 0, 0])

##
#Define Voxel Dimensions 
VoxDims = np.array([1,1,1])

In [ ]:
##
#Estimate different V(t) operators for different maximum amplitudes of the cardiac pulsatility profile

##
#Define Maximum velocity
MaxVel = np.linspace(0, 1.5, num=6)

##
#Initialise Array
V = np.zeros((int(opt['nTR']),MaxVel.shape[0]))
Signal = np.zeros((int(opt['nTR']),MaxVel.shape[0]),dtype = 'c8')

##
#Forward Simulate Velocity & Signal Profiles 
for idx, k in np.ndenumerate(MaxVel):
    Card = np.array([0, 0, k])
    V[:,idx] = np.squeeze(MotionOperator(opt.copy(),Trans,Rot,Card,opt['Mask'][:,np.newaxis,np.newaxis],VoxDims))[:,np.newaxis]
    Signal[:,idx]=EPGMotion(opt, np.squeeze(V[:,idx]))[:,np.newaxis]

In [ ]:
##
#Add Noise

##
#Define SNR Levels
SNR = [10,20,50,np.inf]

##
#Estimate Noise Standard Deviation
NoiseSD = np.zeros((Signal.shape[1],len(SNR)))
for k in range(NoiseSD.shape[0]):
    for l in range(NoiseSD.shape[1]):
        NoiseSD[k,l] = np.mean(abs(Signal[int(opt["nDummy"][0]):,k]))/SNR[l]

##
#Number of Repeats
nRepeats = 10

##
#Initialise SignalNoise Repeats
SignalNoise = np.zeros((*Signal.shape,len(SNR),nRepeats),dtype = 'c8')

##
#Add Noise
for k in range(SignalNoise.shape[1]):
    for l in range(SignalNoise.shape[2]):
        for m in range(SignalNoise.shape[3]):
            SignalNoise[:,k,l,m] = (np.real(Signal[:,k]) + np.random.normal(0, NoiseSD[k,l], Signal.shape[0])) + 1j*(np.imag(Signal[:,k]) + np.random.normal(0, NoiseSD[k,l], Signal.shape[0]))

In [ ]:
##
#Define lower and upper parameter bounds (fix S0 equal to 1, define MotionParameters between -5 and 5 mm/s))
low = [1E-6, 1, -20*np.pi, *np.ones((int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])))*-5]
high = [10E-3, 1+ + np.finfo('f8').eps, 20*np.pi, *np.ones((int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])))*5]

In [ ]:
##
#Fitting without Motion Estimation

##
#Initialise fitting parameters (D, S0, Phi)
par_init = [0.5E-4, 1, np.pi/2]

##
#Initialise Array
poptnoMotion = np.zeros((3,*SignalNoise.shape[1:]))

##
#Perform Fitting
for k in range(SignalNoise.shape[1]):
    for l in range(SignalNoise.shape[2]):
        for m in range(SignalNoise.shape[3]):
            #Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
            Data = np.concatenate((np.real(SignalNoise[int(opt["nDummy"][0]):,k,l,m]).reshape(SignalNoise[int(opt["nDummy"][0]):,k,l,m].shape[0]),np.imag(SignalNoise[int(opt["nDummy"][0]):,k,l,m]).reshape(SignalNoise[int(opt["nDummy"][0]):,k,l,m].shape[0])))
            #Perform Fitting
            poptnoMotion[:,k,l,m], pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low[0:3],high[0:3]),verbose=1,jac=lambda x, *theta: EPGForwardModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact',max_nfev=1E5,ftol=1e-5, xtol=1e-5, gtol=1e-5)


In [ ]:
##
#Fitting with Motion Estimation

##
#Initialise fitting parameters (D, S0, Phi, MotionVector)
par_init = [0.5E-4, 1, np.pi/2, *np.zeros(int(opt["nTR"][0])-int(opt["SteadyStateTR"][0]))]

##
#Initialise Array
poptMotion = np.zeros((len(low),*SignalNoise.shape[1:]))

##
#Perform Fitting
for k in range(SignalNoise.shape[1]):
    for l in range(SignalNoise.shape[2]):
        for m in range(SignalNoise.shape[3]):
            #Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
            Data = np.concatenate((np.real(SignalNoise[int(opt["nDummy"][0]):,k,l,m]).reshape(SignalNoise[int(opt["nDummy"][0]):,k,l,m].shape[0]),np.imag(SignalNoise[int(opt["nDummy"][0]):,k,l,m]).reshape(SignalNoise[int(opt["nDummy"][0]):,k,l,m].shape[0])))
            #Perform Fitting
            poptMotion[:,k,l,m], pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardMotionModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low,high),verbose=1,jac=lambda x, *theta: EPGForwardMotionModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact',max_nfev=1E5,ftol=1e-5, xtol=1e-5, gtol=1e-5)

In [ ]:
##
#Reconstruct Signal & Motion Profile

##
#Create Reconstructed Motion Array
VRecon = np.zeros_like(SignalNoise, dtype = 'f8')
VRecon[-(int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])):,...] = poptMotion[3:,...]

##
#Initialise Signal Array
SignalRecon = np.zeros_like(SignalNoise, dtype = 'c8')

##
#Reconstruct Signal
for k in range(SignalNoise.shape[1]):
    for l in range(SignalNoise.shape[2]):
        for m in range(SignalNoise.shape[3]):
            ##
            #Declare outputs from fitting to the options file 
            optRecon = opt.copy()
            optRecon['D'] = np.asarray([poptMotion[0,k,l,m]], dtype='f8')   
            optRecon['phi'] = np.asarray([poptMotion[2,k,l,m]], dtype='f8') 

            ##
            #Reconstruct signal
            SignalRecon[:,k,l,m] = EPGMotion(optRecon.copy(), VRecon[:,k,l,m])

In [ ]:
##
#Plot Figure
fig, axs = plt.subplots(1, 2)
fig.set_size_inches(8,8)

##
#No Motion Correction
axs[0].imshow(np.mean(poptnoMotion[0,:,:,:]*10**3,axis=-1),clim=[0,4])

##
#Motion Correction
im = axs[1].imshow(np.mean(poptMotion[0,:,:,:]*10**3,axis=-1),clim=[0,4]); plt.colorbar(im, ax = axs, fraction=0.0325, pad=0.025, label = r'D ($\cdot 10^{-3}$ mm$^2$/s)')

##
#Labels & Axes
axs[0].set_xticks([0,1,2,3])
axs[0].set_xticklabels((10,20,50,r'$\infty$'))
axs[0].set_yticks([0,1,2,3,4,5])
axs[0].set_yticklabels((np.round(MaxVel,1)))
axs[0].set_xlabel('SNR')
axs[0].set_ylabel('Max. Pulsatile Velocity (mm/s)')
axs[0].set_title('No Motion Correction')
axs[1].set_xticks([0,1,2,3])
axs[1].set_xticklabels((10,20,50,r'$\infty$'))
axs[1].set_yticks([0,1,2,3,4,5])
axs[1].set_yticklabels((np.round(MaxVel,1)))
axs[1].set_xlabel('SNR')
axs[1].set_title('No Motion Correction')
axs[1].set_title('Motion Corrected (EPG-motion)')

##
#Add Highighted Cell
rect = plt.Rectangle((2.5, -0.53), 1,1, fill=False,linewidth=0.5,color="k")
axs[0].add_patch(rect)
rect = plt.Rectangle((2.5, -0.53), 1,1, fill=False,linewidth=0.5,color="k")
axs[1].add_patch(rect)

In [ ]:
##
#Save Figure
fig.savefig(''.join([OutputPath,'Figure6.png']),dpi=300,format='png',bbox_inches='tight')